In [1]:
import math
import os
import gc
import sys
import time

from typing import List

from numba import jit, njit
from pathlib import Path
from tqdm.notebook import tqdm

In [2]:
BASE_DIR = '/home/dmitry/projects/dfdc'
SRC_DIR = os.path.join(BASE_DIR, 'src')
DATA_DIR = os.path.join(BASE_DIR, 'data/dfdc-videos')
SAVE_DIR = os.path.join(BASE_DIR, 'data/dfdc-crops')

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import cv2

import torch
import torchvision

from torchvision import ops

import nvidia.dali as dali
from nvidia.dali.plugin.pytorch import DALIGenericIterator

# src
sys.path.insert(0, SRC_DIR)
from sample.reader import VideoReader
from dataset.utils import read_labels

# Pytorch_Retinaface
sys.path.insert(0, os.path.join(BASE_DIR, 'Pytorch_Retinaface'))
from data import cfg_mnet
from layers.functions.prior_box import PriorBox
from models.retinaface import RetinaFace
from detect_utils import detect, load_model, postproc_detections
from utils.nms.py_cpu_nms import py_cpu_nms

In [4]:
@njit
def calc_axis(c0, c1, pad, cmax):
    c0 = max(0, c0 - pad)
    c1 = min(cmax, c1 + pad)
    return c0, c1, c1 - c0


@njit
def expand_bbox(bbox, pct):
    bbox = np.copy(bbox)
    bbox[:2] *= 1 - pct
    bbox[2:] *= 1 + pct
    return bbox


@njit
def crop_face(img, bbox, pad_pct=0.05, square=True):
    img_h, img_w, _ = img.shape
    
    if pad_pct > 0:
        bbox = expand_bbox(bbox, pad_pct)
        
    x0, y0, x1, y1 = bbox.astype(np.int16)
    
    if square:
        w, h = x1 - x0, y1 - y0
        if w > h:
            pad = (w - h) // 2
            y0, y1, h = calc_axis(y0, y1, pad, img_h)
        elif h > w:
            pad = (h - w) // 2
            x0, x1, w = calc_axis(x0, x1, pad, img_w)
    
    size = min(w, h)
    face = img[y0:y1, x0:x1][:size, :size]
    return face

In [5]:
def round_num_faces(num_faces, frac_thresh=0.25):
    avg = num_faces.mean()
    fraction, integral = np.modf(avg)
    rounded = integral if fraction < frac_thresh else integral + 1
    return int(rounded)

In [6]:
class VideoPipe(dali.pipeline.Pipeline):
    def __init__(self, file_list: str, seq_len=30, stride=10, 
                 batch_size=1, num_threads=1, device_id=0):
        super(VideoPipe, self).__init__(
            batch_size, num_threads, device_id, seed=3)
        
        self.input = dali.ops.VideoReader(
            device='gpu', file_list=file_list, sequence_length=seq_len, 
            stride=stride, shard_id=0, num_shards=1)

    def define_graph(self):
        output, labels = self.input(name='reader')
        return output, labels
    
    
def get_file_list(df: pd.DataFrame, start: int, end: int, 
                  base_dir: str =DATA_DIR) -> List[str]:
    path_fn = lambda row: os.path.join(base_dir, row.dir, row.name)
    return df.iloc[start:end].apply(path_fn, axis=1).values.tolist()


def write_file_list(files: List[str], path='./file_list.txt') -> None:    
    with open(path, mode='w') as h:
        for i, f in enumerate(files):
            h.write(f'{f} {i}\n')


def init_detector(cfg, weights, use_cpu=False):
    cfg['pretrain'] = False
    net = RetinaFace(cfg=cfg, phase='test')
    net = load_model(net, weights, use_cpu)
    net.eval()
    return net


def mkdirs(base_dir, chunk_dirs):
    for chunk_dir in chunk_dirs:
        dir_path = os.path.join(base_dir, chunk_dir)
        if not os.path.isdir(dir_path):
            os.mkdir(dir_path)

In [7]:
def prepare_imgs_gpu(sample):
    n, h, w, c = sample.shape
    imgs = sample.float()
    imgs -= torch.tensor([104, 117, 123], device=imgs.device)
    imgs = imgs.permute(0, 3, 1, 2)
    scale = torch.tensor([w, h, w, h])
    return imgs, scale


def detect(sample, model, cfg, device):
    bs = cfg['batch_size']
    num_frames, height, width, ch = sample.shape
    imgs, scale = prepare_imgs_gpu(sample)

    priorbox = PriorBox(cfg, image_size=(height, width))
    priors = priorbox.forward().to(device)
    scale = scale.to(device)

    detections = []
    for start in range(0, num_frames, bs):
        end = start + bs
        imgs_batch = imgs[start:end] #.to(device)
        with torch.no_grad():
            loc, conf, landms = model(imgs_batch)
        imgs_batch, landms = None, None
        dets = postproc_detections(loc, conf, priors, scale, cfg)
        detections.append(dets)
        loc, conf = None, None
    return np.vstack(detections) if len(detections) > 1 else detections[0]

In [8]:
a = np.zeros(100, dtype=np.bool)

In [9]:
sys.getsizeof(a), a.nbytes

(196, 100)

In [10]:
a[[1,23,12,13,43,76,78,34]] = True

In [11]:
(~a).nonzero()[0]

array([ 0,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 14, 15, 16, 17, 18, 19,
       20, 21, 22, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 35, 36, 37, 38,
       39, 40, 41, 42, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56,
       57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73,
       74, 75, 77, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92,
       93, 94, 95, 96, 97, 98, 99])

In [12]:
def prepare_data(
        start=0, end=None, chunk_dirs=None, max_open_files=100, 
        file_list_path='./temp_files.txt', verbose=False,
        num_frames=30, stride=10, use_cpu=False, bs=32, 
        base_dir=BASE_DIR, data_dir=DATA_DIR, save_dir=SAVE_DIR):
    df = read_labels(data_dir, chunk_dirs=chunk_dirs)
    mkdirs(save_dir, df['dir'].unique())
    
    device = torch.device("cpu" if use_cpu else "cuda")
    weights_mnet = os.path.join(base_dir, 'data/weights/mobilenet0.25_Final.pth')
    cfg = {**cfg_mnet, 'batch_size': bs}
    detector = init_detector(cfg, weights_mnet, use_cpu).to(device)
    
    if end is None:
        end = len(df)
    
    for start_pos in range(start, end, max_open_files):
        end_pos = min(start_pos + max_open_files, end)
        files = get_file_list(df, start_pos, end_pos)
        write_file_list(files, path=file_list_path)
        pipe = VideoPipe(file_list_path, seq_len=num_frames, stride=stride)
        pipe.build()
        num_samples_read = pipe.epoch_size('reader')
        # run alternative detector if num_samples_read == 0
        run_fallback_reader = num_samples_read < len(files)
        if run_fallback_reader:
            proc_file_idxs = np.zeros(len(files), dtype=np.bool)
        
        data_iter = DALIGenericIterator(
            [pipe], ['images', 'label'], num_samples_read, dynamic_shape=True)
        for idx, video_batch in tqdm(enumerate(data_iter), total=num_samples_read):
            if verbose: 
                t0 = time.time()
            images = video_batch[0]['images'].squeeze(0)
            read_idx =  video_batch[0]['label'].item()
            abs_idx = start_pos + read_idx
            meta = df.iloc[abs_idx]
            # fake = bool(meta['label'])
            sample_dir = os.path.join(save_dir, meta.dir, meta.name[:-4])
            if not os.path.isdir(sample_dir):
                os.mkdir(sample_dir)
            detections = detect(images, detector, cfg_mnet, device)
            images = images.cpu().numpy()
            num_faces = np.array(list(map(len, detections)), dtype=np.uint8)
            max_faces_per_frame = round_num_faces(num_faces, frac_thresh=0.25)
            for f in range(num_frames):
                for det in detections[f][:max_faces_per_frame]:
                    face = crop_face(images[f], det[:4])
                    file_path = os.path.join(sample_dir, '%03d.png' % f)
                    face = cv2.cvtColor(face, cv2.COLOR_RGB2BGR)
                    # cv2.imwrite(file_path, face)
            detections = None
            if run_fallback_reader:
                proc_file_idxs[read_idx] = True
            if verbose:
                t1 = time.time()
                print('[%6d][%.02f s] %s' % (abs_idx, t1 - t0, sample_dir))     
        pipe, data_iter = None, None
        gc.collect()
        if run_fallback_reader:
            unproc_file_idxs = (~proc_file_idxs).nonzero()[0]
            print('Sorry, master, I can\'t handle these:\n')
            for idx in unproc_file_idxs:
                print('[%6d] %s' % (start_pos + idx, files[idx]))
    print('DONE')

In [13]:
!ls ../data/dfdc-videos/dfdc_train_part_49 | wc -l

3135


In [15]:
%%time
gc.collect()
prepare_data(start=2650, end=3000, max_open_files=350, bs=30, verbose=True, 
             chunk_dirs=['dfdc_train_part_49'])

Loading pretrained model from /home/dmitry/projects/dfdc/data/weights/mobilenet0.25_Final.pth
remove prefix 'module.'
Missing keys:0
Unused checkpoint keys:0
Used keys:300


[  2650][0.86 s] /home/dmitry/projects/dfdc/data/dfdc-crops/dfdc_train_part_49/kvyvazqlev
[  2651][0.44 s] /home/dmitry/projects/dfdc/data/dfdc-crops/dfdc_train_part_49/vjnbtebjhv
[  2652][0.45 s] /home/dmitry/projects/dfdc/data/dfdc-crops/dfdc_train_part_49/bpiuysyyuj
[  2653][0.42 s] /home/dmitry/projects/dfdc/data/dfdc-crops/dfdc_train_part_49/zwqdszjdop
[  2654][0.44 s] /home/dmitry/projects/dfdc/data/dfdc-crops/dfdc_train_part_49/cjmfuasyog
[  2655][0.42 s] /home/dmitry/projects/dfdc/data/dfdc-crops/dfdc_train_part_49/szklntqqjo
[  2656][0.44 s] /home/dmitry/projects/dfdc/data/dfdc-crops/dfdc_train_part_49/ixsabuaykm
[  2657][0.43 s] /home/dmitry/projects/dfdc/data/dfdc-crops/dfdc_train_part_49/qvsskinsyi
[  2658][0.42 s] /home/dmitry/projects/dfdc/data/dfdc-crops/dfdc_train_part_49/eifdjwcjnq
[  2659][0.44 s] /home/dmitry/projects/dfdc/data/dfdc-crops/dfdc_train_part_49/rnpzpgncvx
[  2660][0.44 s] /home/dmitry/projects/dfdc/data/dfdc-crops/dfdc_train_part_49/ljcjouvznz
[  2661][0

[  2708] /home/dmitry/projects/dfdc/data/dfdc-videos/dfdc_train_part_49/tfsbglocei.mp4
[  2709] /home/dmitry/projects/dfdc/data/dfdc-videos/dfdc_train_part_49/sfmhpvahzi.mp4
[  2710] /home/dmitry/projects/dfdc/data/dfdc-videos/dfdc_train_part_49/xgygzngqvi.mp4
[  2711] /home/dmitry/projects/dfdc/data/dfdc-videos/dfdc_train_part_49/gdwbzayezx.mp4
[  2712] /home/dmitry/projects/dfdc/data/dfdc-videos/dfdc_train_part_49/vgrscvxomm.mp4
[  2713] /home/dmitry/projects/dfdc/data/dfdc-videos/dfdc_train_part_49/rtiyduycrn.mp4
[  2714] /home/dmitry/projects/dfdc/data/dfdc-videos/dfdc_train_part_49/pchdcgnyri.mp4
[  2715] /home/dmitry/projects/dfdc/data/dfdc-videos/dfdc_train_part_49/azkhkminwu.mp4
[  2716] /home/dmitry/projects/dfdc/data/dfdc-videos/dfdc_train_part_49/dbkrfxlell.mp4
[  2717] /home/dmitry/projects/dfdc/data/dfdc-videos/dfdc_train_part_49/nsysyqeexx.mp4
[  2718] /home/dmitry/projects/dfdc/data/dfdc-videos/dfdc_train_part_49/uhksnblalh.mp4
[  2719] /home/dmitry/projects/dfdc/data/df